In [1]:
# =========================================================
# セル0: プロジェクトルートを sys.path に追加
# =========================================================
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
sys.path.append(PROJECT_ROOT)

print("Added to sys.path:", PROJECT_ROOT)


Added to sys.path: c:\Users\AVRJ0006\YamatoDX\YAMATO_learning_practice\my_time_series_benchmark\project_git\My-Time-Series-Benchmark\mtsb


In [2]:
# =========================================================
# セル1: インポート（naive.ipynb と同じ構造）
# =========================================================
import pandas as pd
import numpy as np
from pathlib import Path

# モデル
from models.baseline.moving_average import MovingAverageModel

# 指標
from evaluation.basic.mae import MAE
from evaluation.basic.rmse import RMSE
from evaluation.basic.mape import MAPE
from evaluation.basic.smape import SMAPE
from evaluation.basic.r2 import R2


In [3]:
# =========================================================
# セル2: データ読み込み（パスも naive.ipynb と完全一致）
# =========================================================
df = pd.read_csv("../data/raw/daily_sales.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")

y = df["sales"]

horizon = 30

train = y.iloc[:-horizon]
test = y.iloc[-horizon:]

train.head(), test.head()


(date
 2023-01-01    127.94
 2023-01-02    133.14
 2023-01-03    129.73
 2023-01-04    122.88
 2023-01-05    124.41
 Name: sales, dtype: float64,
 date
 2023-06-20    133.05
 2023-06-21    134.51
 2023-06-22    125.49
 2023-06-23    130.29
 2023-06-24    133.10
 Name: sales, dtype: float64)

In [4]:
# =========================================================
# セル3: Moving Average モデルの学習 & 予測
# =========================================================

window = 7  # 任意（例）

model = MovingAverageModel(freq="D", window=window)
model.fit(train)

# naive.ipynb と同じく forecast() ではなく y_pred() を呼ぶインターフェース
y_pred = model.forecast(horizon)

y_pred.head()


2023-06-20    131.765714
2023-06-21    131.765714
2023-06-22    131.765714
2023-06-23    131.765714
2023-06-24    131.765714
Freq: D, Name: moving_average, dtype: float64

In [5]:
# =========================================================
# セル4: 評価指標（順番も naive.ipynb と完全一致）
# =========================================================
metrics = {
    "mae": MAE().compute(test, y_pred),
    "rmse": RMSE().compute(test, y_pred),
    "mape": MAPE().compute(test, y_pred),
    "smape": SMAPE().compute(test, y_pred),
    "r2": R2().compute(test, y_pred),
}

metrics


{'mae': 12.133333333333347,
 'rmse': 15.030785021659621,
 'mape': 8.183461502640114,
 'smape': 8.654524547784122,
 'r2': -0.7117554706541853}

In [6]:
# =========================================================
# セル5: horizon 別誤差（naive.ipynb と同じ構造）
# =========================================================
horizon_rows = []
for h in range(1, horizon + 1):
    yt = test.iloc[h-1]
    yp = y_pred.iloc[h-1]

    horizon_rows.append({
        "horizon": h,
        "abs_error": abs(yt - yp),
        "squared_error": (yt - yp)**2,
        "pct_error": abs(yt - yp)/abs(yt) * 100 if yt != 0 else None,
    })

horizon_df = pd.DataFrame(horizon_rows)
horizon_df.head()


,horizon,abs_error,squared_error,pct_error
0,1,1.284286,1.649390,0.965265
1,2,2.744286,7.531104,2.040209
2,3,6.275714,39.384590,5.000968
3,4,1.475714,2.177733,1.132638
4,5,1.334286,1.780318,1.002469


In [7]:
# =========================================================
# セル6: CSV 出力（ファイル名だけ moving_average に変更）
# =========================================================


# forecast
forecast_df = pd.DataFrame({
    "date": y_pred.index,
    "actual": test.values,
    "forecast": y_pred.values,
    "model": "moving_average",
})
forecast_df.to_csv("../../exports_for_tableau/moving_average_forecast.csv", index=False)

# metrics
metrics_df = pd.DataFrame([{
    "model": "moving_average",
    **metrics
}])
metrics_df.to_csv("../../exports_for_tableau/moving_average_metrics.csv", index=False)

# horizon error
horizon_df.to_csv("../../exports_for_tableau/moving_average_horizon_error.csv", index=False)

print("CSV files exported successfully!")


CSV files exported successfully!
